Goal: apply a DL RNN model on different dataset
- Preprocess with MinMaxScaler
- Preprocess with CustomerTransformer

# Imports, variables, functions

In [1]:
import scipy.io
from pyedflib import highlevel
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from itertools import groupby
import csv
import pickle
from scipy.signal import butter, sosfilt, sosfiltfilt, sosfreqz
from scipy.signal import freqz, iirnotch, filtfilt
from sklearn.preprocessing import MinMaxScaler
from sklearn.base import TransformerMixin, BaseEstimator
import random
from sklearn.model_selection import cross_validate
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from scipy.fftpack import rfft
from sklearn.metrics import accuracy_score
import tensorflow as ts
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from plot_keras_history import plot_history

2022-12-12 20:23:50.652159: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 20:23:51.407817: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-12-12 20:23:51.620285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-12 20:23:51.620314: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [5]:
sample_rate = sampling_rate = 256
sec = 10
len_window = sample_rate*sec
overlap = 5

# Load annotation file
annt = scipy.io.loadmat('../raw_data/annotations_2017.mat')

In [6]:
## -- PREPROCESSING FUNCTIONS --

# Highpass filter
def highpass_filter(signals, sampling_rate, hp_frequency = 0.1):
    sos = butter(N = 3, Wn = hp_frequency, btype="highpass",fs=sampling_rate, output="sos")
    filter_hp = sosfiltfilt(sos, signals)
    return filter_hp

# Powerline filter
def notch_filter(signals, sampling_rate, notch_frequency = 50, quality_factor = 30):
    w0 = notch_frequency/(sampling_rate/2)
    b_notch, a_notch = iirnotch(w0, quality_factor)
    filter_notch = filtfilt(b_notch, a_notch, signals, axis = -1)
    return filter_notch

# Create our own scaler
class CustomTranformer(TransformerMixin, BaseEstimator): 
    # BaseEstimator generates the get_params() and set_params() methods that all Pipelines require
    # TransformerMixin creates the fit_transform() method from fit() and transform()
    
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        self.means = X.mean()
        return self
    
    def transform(self, X, y=None):
        norm_features = X - self.means
        return norm_features

# Combination of all filters and Scaler
def filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30):
    filter_hp = highpass_filter(signals, sampling_rate)
    filter_notch = notch_filter(filter_hp, sampling_rate, notch_frequency, quality_factor)
    final_signal = scaler.fit_transform(filter_notch)
    return final_signal

In [7]:
## -- LABEL FUNCTIONS --

# Format the EEG 
def eeg_formated(signals, names_ele):
    data_signals = signals.T # transpose the signals from datapoints
    data_signals = pd.DataFrame(data_signals) # create a pandas dataframe
    
    data_signals.columns = names_ele # rename columns
    
    return data_signals

# Format the annotations
def diagnosis(n):
    patient_A=annt["annotat_new"][0][n-1][0]
    patient_B=annt["annotat_new"][0][n-1][1]
    patient_C=annt["annotat_new"][0][n-1][2]
    
    #converting seconds to datapoints

    patient_A=patient_A.tolist()
    patient_B=patient_B.tolist()
    patient_C=patient_C.tolist()
    
    patient_A_dtp=[]
    patient_B_dtp=[]
    patient_C_dtp=[]  
    for elem in patient_A:
        for i in range(sampling_rate):
            patient_A_dtp.append(elem) 
    for elem in patient_B:
        for i in range(sampling_rate):
            patient_B_dtp.append(elem)
        
    for elem in patient_C:
        for i in range(sampling_rate):
            patient_C_dtp.append(elem)
            
    target_=pd.DataFrame({"Diagnosis A":patient_A_dtp,"Diagnosis B":patient_B_dtp,"Diagnosis C":patient_C_dtp})
    
    return target_  

# Add a time column with the seconds
def add_time(df):
    list_time=[]
    for i in range(len(df)):
        list_time.append(i//sampling_rate)
    df["time"]=list_time
    return df

# Create target variables when seizures lasts at least 10
def is_seizure(df):
    
    threshold = sampling_rate*10
    
    df['is_seizure_A'] = df["Diagnosis A"].groupby((df["Diagnosis A"] != df["Diagnosis A"].shift()).cumsum()).transform('size') * df["Diagnosis A"]
    df['is_seizure_A'] = (df['is_seizure_A'] > threshold).astype(int)
    
    df['is_seizure_B'] = df["Diagnosis B"].groupby((df["Diagnosis B"] != df["Diagnosis B"].shift()).cumsum()).transform('size') * df["Diagnosis B"]
    df['is_seizure_B'] = (df['is_seizure_B'] > threshold).astype(int)
    
    df['is_seizure_C'] = df["Diagnosis C"].groupby((df["Diagnosis C"] != df["Diagnosis C"].shift()).cumsum()).transform('size') * df["Diagnosis C"]
    df['is_seizure_C'] = (df['is_seizure_C'] > threshold).astype(int)
    
    return df 

# Create final target
def create_target(df):
    df['is_seizure_target'] = np.where(df['is_seizure_A'] + df['is_seizure_B'] + df['is_seizure_C'] >= 2, 1, 0)
    return df

# Remove useless
def remove_useless_columns(df):
    df.drop(columns=['Diagnosis A', 'Diagnosis B', 'Diagnosis C', 'is_seizure_A', 'is_seizure_B', 'is_seizure_C', 'ECG EKG', 'Resp Effort', 'time'], inplace=True)
    return df

# Final function to label
def label_data(path_raw_data, signals_preprocessed, n):
    
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    names_ele = [signal_headers[iele]['label'] for iele in range(signals.shape[0])] # extract electrode names
    
    eeg_patient = eeg_formated(signals_preprocessed, names_ele) # format the ECG
    eeg_patient.rename(columns={'ECG EKG-REF':'ECG EKG', 'Resp Effort-REF':'Resp Effort'}, inplace=True)
    diagnosis_patient = diagnosis(n) # format the diagnosis
    
    data_patient = pd.merge(left=eeg_patient, right=diagnosis_patient, how='left', left_index=True, right_index=True) # merge ecg and diagnosis
    
    add_time(data_patient)
    is_seizure(data_patient)
    create_target(data_patient)
    remove_useless_columns(data_patient)
    
    return data_patient

In [8]:
def create_data_input(df):
    data = np.array((df.iloc[i:i+len_window+1]) for i in range(0,len(df)-len_window, overlap*sample_rate))
    r=data.shape[0]
    c=data.shape[1]
    data = pd.DataFrame(data.reshape(r,c))
    
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1]
    return X,y

def oversampling(X, y): 
    sm = SMOTE(sampling_strategy='minority', random_state=7)
    X, y = sm.fit_resample(X, y)
    return X, y

In [9]:
def model_pipeline(path_raw_data, scaler, patient_number, Fournier=False):
    
    # Load raw data
    signals, signal_headers, header = highlevel.read_edf(path_raw_data)
    
    # Preprocess data 
    signals_preprocessed = filter_signals(signals, sampling_rate, scaler, hp_frequency = 0.1, notch_frequency = 50, quality_factor = 30)
    
    if Fournier == True:
        signals_preprocessed = pd.DataFrame(np.array([abs(rfft(signals_preprocessed[i])) for i in range(len(signals_preprocessed))]))
        
    # Label data
    df = label_data(path_raw_data, signals_preprocessed, patient_number)
    
    return df

In [10]:
df = model_pipeline("../raw_data/eeg5.edf",  MinMaxScaler(), 5)

In [11]:
df

,EEG Fp1-Ref,EEG Fp2-Ref,EEG F7-Ref,EEG F3-Ref,EEG Fz-Ref,EEG F4-Ref,EEG F8-Ref,EEG T3-Ref,EEG C3-Ref,EEG Cz-Ref,EEG C4-Ref,EEG T4-Ref,EEG T5-Ref,EEG P3-Ref,EEG Pz-Ref,EEG P4-Ref,EEG T6-Ref,EEG O1-Ref,EEG O2-Ref,is_seizure_target
0,0.503096,0.757229,0.000000,0.300549,0.388281,0.166560,1.000000,0.169898,0.529436,0.612799,0.556498,0.928607,0.271835,0.454323,0.653375,0.663144,0.974370,0.348322,0.681871,1
1,0.308037,0.896578,0.474134,0.360279,0.265175,0.000000,0.973154,0.328420,0.087946,0.559496,0.709583,1.000000,0.147772,0.200096,0.331954,0.720204,0.884000,0.013864,0.344023,1
2,0.162636,0.949387,0.356073,0.088600,0.178670,0.000000,0.973011,0.260397,0.060259,0.507191,0.662359,1.000000,0.106264,0.148687,0.315574,0.720373,0.912142,0.014164,0.397629,1
3,0.262964,0.759260,0.405419,0.266965,0.184892,0.012968,0.961680,0.273960,0.026392,0.502536,0.681395,1.000000,0.107487,0.154859,0.303968,0.708926,0.890831,0.000000,0.362017,1
4,0.390654,0.716113,0.395316,0.266363,0.179682,0.000000,0.928876,0.117525,0.051225,0.502856,0.652177,1.000000,0.066880,0.165627,0.321876,0.716678,0.892062,0.000719,0.358651,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983291,0.144545,0.107801,0.242918,0.081092,0.000000,0.002430,0.198973,0.404907,0.290127,0.130825,0.396163,0.364060,0.733012,0.743284,0.838875,1.000000,0.752451,0.999487,0.800623,0
983292,0.148939,0.125041,0.242304,0.078868,0.000000,0.008518,0.202915,0.389808,0.287819,0.131405,0.399925,0.371192,0.728702,0.740282,0.836054,0.999414,0.771990,1.000000,0.800197,0
983293,0.153174,0.149668,0.242256,0.085214,0.000000,0.006540,0.202562,0.383956,0.289466,0.129266,0.396953,0.332371,0.731896,0.743887,0.836441,0.998724,0.774037,1.000000,0.798906,0
983294,0.380881,0.378804,0.447666,0.337911,0.270305,0.269078,0.414756,0.560082,0.484579,0.363000,0.555702,0.485706,0.810163,0.818057,0.883693,0.999781,0.820379,1.000000,0.853106,0


In [12]:
def redefine_window_target(window_df):
    if len(np.unique(window_df.iloc[:,-1])) == 1:
        target = window_df.iloc[0,-1]
    elif np.unique(window_df.iloc[:,-1],return_counts=True)[1][1] >= 2*256:
        target = 1
    else:
        target = 0
    t_df = window_df.drop(columns = "is_seizure_target")
    window = pd.DataFrame(np.array(t_df))
    window["Target"] = target
    return window

In [13]:
data = np.array([redefine_window_target(df.iloc[i:i+len_window ]) for i in range(0,len(df)-len_window, overlap*sample_rate)])
data.shape

(767, 2560, 20)

In [ ]:
data

In [ ]:
#r=data.shape[0]*data.shape[1]
#c=data.shape[2]
    
#data = pd.DataFrame(data.reshape(r,c))
X = data[:,:,:-1]
y = data[:,:,-1]
y=y.sum(axis=1)>=2#*256
y = y.astype(int)
index_seizure = []
curr = 0
for i, el in enumerate(y):
    if curr == 0 and el == 1:
        index_seizure.append(i)
    curr = el
past_length = 3
for ind in index_seizure:
    y[ind-past_length:ind] = 2

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [1, 1, 1],
       [1, 1, 1],
       [0, 0, 0],
       [0, 0, 0]])

In [63]:
y=y.sum(axis=1)>=2#*256

In [64]:
y = y.astype(int)

In [65]:
y

array([0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0])

In [66]:
index_seizure = []
curr = 0
for i, el in enumerate(y):
    if curr == 0 and el == 1:
        index_seizure.append(i)
    curr = el

In [67]:
index_seizure

[4, 9]

In [69]:
past_length = 2
for ind in index_seizure:
    y[ind-past_length:ind] = 2

In [70]:
y

array([0, 0, 2, 2, 1, 1, 0, 2, 2, 1, 1, 0, 0])

In [ ]:
def oversampling(X, y): 
    sm = SMOTE(sampling_strategy='minority', random_state=7)
    X, y = sm.fit_resample(X, y)
    return X, y

In [ ]:
train_ind, test_ind = train_test_split(np.arange(767), test_size=0.2)

X_train = X[train_ind,:,:]
X_test = X[test_ind,:,:]
y_train = y[train_ind]
y_test = y[test_ind]

#X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
    
    # Balancing

#X_train, y_train = oversampling(X_train, y_train)
#y_train.transpose()

In [ ]:
y_train

In [ ]:
X_train.shape #n.reshape((767, 2561, 19))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential()
model.add(layers.SimpleRNN(units=10, activation='tanh',input_shape=(2561, 19)))
model.add(layers.Dense(10, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))

# The compilation
model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy', 
        metrics=[ts.keras.metrics.Recall(),"accuracy"])

# The fit
model.fit(X_train, y_train,
         batch_size=32,
         epochs=50, verbose=0)

In [ ]:
results_DL =model.evaluate(X_test, y_test)
loss=results_DL[0]
recall = results_DL[1]
accuracy = results_DL[2]
print(loss,recall,accuracy)